# Dataset Preprocessing

Data is preprocessed such that the XML data is converted to CSV form which can be easily accessed using pandas dataframes. 

Final data is stored in the following form:

- Files in the `preprocessed_datasets/sentences` directory contain basic information for each sentence including `rewiev_id`, `sentence_id`, `text` with `sentence_id` as the primary key.
- Files in the `preprocessed_datasets/opinions` directory contain information regarding each opinion for every sentence including `setence_id`, `opinion_id`, `target`, `category`, `polarity`, `from`, `to` with `opinion_id` as the primary key.

In [34]:
!pip install -q transformers
!pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 248.0 kB/s eta 0:00:00m eta 0:00:010:00:01


In [35]:
import pandas as pd
import xml.etree.ElementTree as ET

Defining file locations

In [116]:
dirc = "./datasets/"
dircs = "./preprocessed_datasets/sentences/"
dirco = "./preprocessed_datasets/opinions/"

files = [
    "arabic-hotels",
    "chinese-cameras",
    "chinese-phones",
    "dutch-phones",
    "dutch-restaurants",
    "english-laptops",
    "english-restaurants",
    "french-restaurants",
    "russian-restaurants",
    "spanish-restaurants",
    "turkish-restaurants"
]

xml = ".xml"
csv = ".csv"

Processing data into new dataframes

In [117]:
sdfs = []
odfs = []
for i in range(len(files)):
    tree = ET.parse(dirc + files[i] + xml)
    root = tree.getroot()

    sentences = []
    opinions = []

    for elem in root:
        rid = elem.attrib['rid']

        for sntnc in elem[0]:
            sentence = {}
            sentence["rid"] = rid
            sentence["id"] = sntnc.attrib['id']
            sentence["text"] = sntnc.find('text').text
            sentences.append(sentence)

            if len(sntnc) > 1:
                for i in range(len(sntnc[1])):
                    op = sntnc[1][i]
                    opinion = {}
                    opinion["sid"] = sentence["id"]
                    opinion["id"] = sentence["id"] + ":" + str(i)
                    for key in op.attrib.keys():
                        opinion[key] = op.attrib[key]
                    opinions.append(opinion)

    sdf = pd.DataFrame(sentences)
    odf = pd.DataFrame(opinions)
    sdfs.append(sdf)
    odfs.append(odf)

Saving the data to files

In [118]:
for i in range(len(sdfs)):
    sdf = sdfs[i]
    odf = odfs[i]
    sdf.to_csv(dircs + files[i] + csv, index=False)
    odf.to_csv(dirco + files[i] + csv, index=False)